# Kết nối Drive

# Đọc dữ liệu

In [1]:
import pandas as pd

file_name = "/kaggle/input/hahaha/Thoi_trang_phong_cach.csv"
df = pd.read_csv(file_name, encoding="utf-8-sig")

In [2]:
df = df.drop(df.columns[3], axis=1)
print(df.head(3))

     Tài liệu                                           Nội dung  \
0  Tài liệu 1  ngưỡng U40, thành_viên SNSD duy_trì nhan_sắc t...   
1  Tài liệu 3  đây, sưu_tầm móc khoá Labubu trở_thành trào_lư...   
2  Tài liệu 4  1. Pháo 810 điểm Pháo ghi_điểm kết_hợp trang_p...   

                                                Tags  
0  U40,Tiffany,trang phục,tham khảo,dệt kim,trẻ t...  
1  Labubu,Huyền My,Lisa,BlackPink,Á hậu,đồ chơi,S...  
2  Lọ Lem,Lâm Minh,Ciin,xuyên thấu,húi cua,tạo hì...  


# Hàm xóa stop words, số, loại bỏ từ đơn

In [3]:
with open('/kaggle/input/hahaha/StopWord.txt', 'r', encoding='utf-8') as f:
  vietnamese_stopwords = [word.strip() for word in f.readlines() if word.strip()]

In [4]:
import re

def preprocess_text(text, stopwords):
    text = text.lower()
    cleaned_text = re.sub(r' [^a-zA-Z0-9\sÀ-ỹ|] ', ' ', text)
    cleaned_text = re.sub(r'[^a-zA-Z0-9\sÀ-ỹ|] ', ' ', cleaned_text)

    cleaned_text = re.sub(r'\|([^|]+)\|', r'\1', cleaned_text)

    cleaned_text = re.sub(r'\s{2,}', ' ', cleaned_text).strip()
    cleaned_text = re.sub(r'\b\d+\b', '', cleaned_text)
    cleaned_text = re.sub(r'[^a-zA-Z0-9\sÀ-ỹ|] ', ' ', cleaned_text)
    cleaned_text = re.sub(r'[,]', ' ', cleaned_text)

    words = cleaned_text.split()
    processed_words = [word for word in words if word not in stopwords]

    return ' '.join(processed_words)

In [5]:
preprocess_text("4 tp.H.C.M phát_động, tuy_nhiên thi Đại_sứ văn_hoá đọc 2024 thu_hút 1,6 triệu dự thi học_sinh, sinh_viên 9,2 ngàn cơ_sở giáo_dục nước tham_gia. 517 giải cao vòng sơ_khảo 60 tỉnh thành gửi Ban tổ_chức. Kết_quả, Ban tổ_chức trao 4 danh_hiệu Đại_sứ văn_hoá đọc tiêu_biểu 136 giải nhất, nhì, ba, khuyến_khích giải chuyên_đề thí_sinh tham_gia. đó, Đồng_Nai đoạt 2 giải khuyến_khích Cù_Đặng_Bảo_Trân, lớp 84, Trường TH-THCS-THPT Song_ngữ Á_Châu Nguyễn_Thái_Bình, lớp 12C1, Trường trung_học_phổ_thông Lê_Quý_Đôn Tân_Mai, thành_phố Biên_Hoà", vietnamese_stopwords)

'tp.h.c.m phát_động thi đại_sứ văn_hoá đọc thu_hút triệu dự thi học_sinh sinh_viên ngàn cơ_sở giáo_dục nước tham_gia giải cao vòng sơ_khảo tỉnh thành gửi ban tổ_chức kết_quả ban tổ_chức trao danh_hiệu đại_sứ văn_hoá đọc tiêu_biểu giải nhì khuyến_khích giải chuyên_đề thí_sinh tham_gia đồng_nai đoạt giải khuyến_khích cù_đặng_bảo_trân lớp trường th-thcs-thpt song_ngữ á_châu nguyễn_thái_bình lớp 12c1 trường trung_học_phổ_thông lê_quý_đôn tân_mai thành_phố biên_hoà'

# Chuẩn bị dữ liệu để nạp vào mô hình SVM

In [6]:
df['Nội dung'] = df['Nội dung'].apply(lambda x: preprocess_text(x, vietnamese_stopwords))
print(df['Nội dung'])

0       ngưỡng u40 thành_viên snsd duy_trì nhan_sắc tr...
1       sưu_tầm móc khoá labubu trào_lưu giới trẻ lisa...
2       pháo điểm pháo ghi_điểm kết_hợp trang_phục ấn_...
3       câu lụa đẹp người hồ_ngọc_hà diện item cơ_bản ...
4       mùa thu quần_áo váy tủ giày chị_em cập_nhật th...
                              ...                        
4103    bật hiệu_ứng sạc pin điện_thoại vivo làm_đẹp s...
4104    hoa_hậu thuỳ_tiên màn ấn_tượng họp_báo công_bố...
4105    máy_bay đỗ nhà dân quanh sân_bay đà_nẵng dễ_dà...
4106    new_york_post tôm_hùm clementine mắc lưới tôm_...
4107    hãng truyền_thông địa_phương phi lợi_nhuận mai...
Name: Nội dung, Length: 4108, dtype: object


In [7]:
!pip3 install pyvi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 53.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.3 MB/s eta 0:00:00


In [8]:
import pandas as pd
import numpy as np
from pyvi import ViTokenizer, ViPosTagger
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

def preprocess_and_extract_pos(tokens):
    _, pos_tags = ViPosTagger.postagging(" ".join(tokens))
    return pos_tags

pos_tag = df['Nội dung'].apply(preprocess_and_extract_pos)

In [9]:
import json

df['POS_Tags'] = pos_tag.apply(lambda x: x.split(',') if isinstance(x, str) else x)
df['POS_Tags'] = df['POS_Tags'].apply(lambda x: json.dumps(x) if isinstance(x, list) else x)

In [10]:
print(df.head())

     Tài liệu                                           Nội dung  \
0  Tài liệu 1  ngưỡng u40 thành_viên snsd duy_trì nhan_sắc tr...   
1  Tài liệu 3  sưu_tầm móc khoá labubu trào_lưu giới trẻ lisa...   
2  Tài liệu 4  pháo điểm pháo ghi_điểm kết_hợp trang_phục ấn_...   
3  Tài liệu 5  câu lụa đẹp người hồ_ngọc_hà diện item cơ_bản ...   
4  Tài liệu 6  mùa thu quần_áo váy tủ giày chị_em cập_nhật th...   

                                                Tags  \
0  U40,Tiffany,trang phục,tham khảo,dệt kim,trẻ t...   
1  Labubu,Huyền My,Lisa,BlackPink,Á hậu,đồ chơi,S...   
2  Lọ Lem,Lâm Minh,Ciin,xuyên thấu,húi cua,tạo hì...   
3  Quế Anh,Mai Ngô,Carpet Check,lăng xê,Miss Gran...   
4  mùa thu,mule,Jane,sành điệu,sắm,chị em,giày,xu...   

                                            POS_Tags  
0  [N, V, N, V, N, V, N, V, N, M, M, Nu, N, N, V,...  
1  [N, N, A, F, N, V, N, A, V, N, N, V, N, V, N, ...  
2  [N, V, N, V, N, V, N, M, N, V, N, V, N, V, N, ...  
3  [N, N, V, N, V, N, V, N, V, N, 

In [11]:
# output_file_path = '/content/processed_dataframe.csv'
# df.to_csv(output_file_path, index=False, encoding='utf-8-sig')

# output_file_path

In [12]:
df['Tags'] = df['Tags'].str.lower()
df['Tags_split'] = df['Tags'].apply(lambda x: set(x.split(',')) if isinstance(x, str) else set())
df['Tags_split'] = df['Tags_split'].apply(
    lambda tags: {tag.replace(' ', '_') for tag in tags} if isinstance(tags, set) else tags
)

def prepare_training_data(row):
    tokens = row['Nội dung'].split()
    labels = [1 if token in row['Tags_split'] else 0 for token in tokens]
    return labels

df['Labels'] = df.apply(prepare_training_data, axis=1)

In [13]:
print(df.head())
tmp_df = df

     Tài liệu                                           Nội dung  \
0  Tài liệu 1  ngưỡng u40 thành_viên snsd duy_trì nhan_sắc tr...   
1  Tài liệu 3  sưu_tầm móc khoá labubu trào_lưu giới trẻ lisa...   
2  Tài liệu 4  pháo điểm pháo ghi_điểm kết_hợp trang_phục ấn_...   
3  Tài liệu 5  câu lụa đẹp người hồ_ngọc_hà diện item cơ_bản ...   
4  Tài liệu 6  mùa thu quần_áo váy tủ giày chị_em cập_nhật th...   

                                                Tags  \
0  u40,tiffany,trang phục,tham khảo,dệt kim,trẻ t...   
1  labubu,huyền my,lisa,blackpink,á hậu,đồ chơi,s...   
2  lọ lem,lâm minh,ciin,xuyên thấu,húi cua,tạo hì...   
3  quế anh,mai ngô,carpet check,lăng xê,miss gran...   
4  mùa thu,mule,jane,sành điệu,sắm,chị em,giày,xu...   

                                            POS_Tags  \
0  [N, V, N, V, N, V, N, V, N, M, M, Nu, N, N, V,...   
1  [N, N, A, F, N, V, N, A, V, N, N, V, N, V, N, ...   
2  [N, V, N, V, N, V, N, M, N, V, N, V, N, V, N, ...   
3  [N, N, V, N, V, N, V, N, V,

In [14]:
from tqdm import tqdm

new_data = []

for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing rows"):
    tokens = row['Nội dung'].split()
    pos_tags = row['POS_Tags']
    for word, pos in zip(tokens, pos_tags):
        label = 1 if word in row['Tags_split'] else 0
        new_entry = {'idx': index, 'Từ': word, 'POS Tag': pos, 'Labels': label}
        new_data.append(new_entry)

new_df = pd.DataFrame(new_data)

print(new_df.head())

Processing rows: 100%|██████████| 4108/4108 [00:04<00:00, 938.96it/s] 


   idx          Từ POS Tag  Labels
0    0      ngưỡng       N       0
1    0         u40       V       1
2    0  thành_viên       N       0
3    0        snsd       V       0
4    0     duy_trì       N       0


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Nội dung'])

In [16]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

terms = vectorizer.get_feature_names_out()

dense_matrix = X.toarray()
tfidf_scores = dense_matrix.sum(axis=0)

tfidf_df = pd.DataFrame({'Term': terms, 'TF-IDF': tfidf_scores}).sort_values(by='TF-IDF', ascending=False)
tfidf_df

,Term,TF-IDF
20408,người,140.739428
34143,xe,119.876279
14543,không,113.907383
13520,iphone,90.830907
20703,nhà,85.757991
...,...,...
6798,côngthổ,0.014072
10487,giờ_hoàng_đạo,0.014072
6612,cát_tinh_cao_chiếu,0.014072
19911,nguy_nan,0.014072


In [17]:
df = tmp_df
tmp_new_df = new_df

In [18]:
print(new_df.shape)

(1069817, 4)


In [19]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

tfidf_df.rename(columns={'Term': 'Từ'}, inplace=True)

label_encoder = LabelEncoder()

merged_df = pd.merge(new_df, tfidf_df, on='Từ', how='left')
merged_df.rename(columns={'TF-IDF': 'TF-IDF Value'}, inplace=True)
merged_df = merged_df.dropna(subset=['TF-IDF Value'])

scaler = MinMaxScaler()
merged_df['TF-IDF Value'] = scaler.fit_transform(merged_df[['TF-IDF Value']])

one_hot_encoder = OneHotEncoder(sparse=False)
encoded_pos = one_hot_encoder.fit_transform(merged_df[['POS Tag']])

pos_columns = one_hot_encoder.get_feature_names_out(['POS Tag'])  # Lấy tên các cột sau encoding
encoded_pos_df = pd.DataFrame(encoded_pos, columns=pos_columns, index=merged_df.index)

merged_df = pd.concat([merged_df, encoded_pos_df], axis=1)

merged_df = merged_df.drop(columns=['POS Tag'])

columns_to_keep = ['idx', 'Từ', 'TF-IDF Value', 'Labels', 'POS Tag_N', 'POS Tag_V']
merged_df = merged_df[columns_to_keep]

labels = merged_df.pop('Labels')
merged_df['Labels'] = labels

print(merged_df)

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


          idx          Từ  TF-IDF Value  POS Tag_N  POS Tag_V  Labels
0           0      ngưỡng      0.010582        1.0        0.0       0
1           0         u40      0.005998        0.0        1.0       1
2           0  thành_viên      0.091738        1.0        0.0       0
3           0        snsd      0.001853        0.0        1.0       0
4           0     duy_trì      0.112928        1.0        0.0       0
...       ...         ...           ...        ...        ...     ...
1069811  4107     lan_toả      0.023215        0.0        1.0       0
1069812  4107   ưa_chuộng      0.073891        1.0        0.0       0
1069813  4107    thế_giới      0.182701        0.0        0.0       0
1069814  4107        loài      0.076173        1.0        0.0       0
1069815  4107    sinh_vật      0.016872        1.0        0.0       0

[1066142 rows x 6 columns]


In [20]:
label_counts = merged_df['Labels'].value_counts(normalize=True) * 100

label_distribution = pd.DataFrame({
    'Label': label_counts.index,
    'Percentage': label_counts.values
})

print(label_distribution)

   Label  Percentage
0      0   86.073337
1      1   13.926663


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

X = train_columns = merged_df.iloc[:, 2:-1]
y = merged_df['Labels'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

class_weights = {0: 1 - (86 / 100), 1: 1 - (14 / 100)}

svm_model = SVC(kernel='linear', class_weight=class_weights)
svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Độ chính xác của mô hình: {accuracy:.2f}')

report = classification_report(y_test, y_pred, target_names=["Non-Keyword", "Keyword"])
print(report)